<a href="https://colab.research.google.com/github/IM2COLD/MachineLearningWithKaggle/blob/main/llie_mirnet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install numpy opencv-python torchmetrics torchvision tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 17.7 MB/s eta 0:00:00


In [ ]:
!gdown https://drive.google.com/uc?id=1DdGIJ4PZPlF2ikl8mNM9V-PdVxVLbQi6
!gdown https://drive.google.com/uc?id=1dzuLCk9_gE2bFF222n3-7GVUlSVHpMYC
!mkdir -p dataset
!unzip -oq -d dataset lol_dataset.zip
!unzip -oq -d dataset LOL-v2.zip

Downloading...
From: https://drive.google.com/uc?id=1DdGIJ4PZPlF2ikl8mNM9V-PdVxVLbQi6
To: /content/lol_dataset.zip
100% 347M/347M [00:01<00:00, 181MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dzuLCk9_gE2bFF222n3-7GVUlSVHpMYC
To: /content/LOL-v2.zip
100% 1.05G/1.05G [00:04<00:00, 222MB/s]


In [ ]:
import cmath
import cv2
import google
#import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torchmetrics
import torchvision
from tqdm import tqdm

In [ ]:
## Learning Enriched Features for Fast Image Restoration and Enhancement
## Syed Waqas Zamir, Aditya Arora, Salman Khan, Munawar Hayat, Fahad Shahbaz Khan, Ming-Hsuan Yang, and Ling Shao
## IEEE Transactions on Pattern Analysis and Machine Intelligence (TPAMI)
## https://www.waqaszamir.com/publication/zamir-2022-mirnetv2/

##########################################################################
##---------- Selective Kernel Feature Fusion (SKFF) ----------
class SKFF(torch.nn.Module):
    def __init__(self, in_channels, height=3,reduction=8,bias=False):
        super(SKFF, self).__init__()

        self.height = height
        d = max(int(in_channels/reduction),4)

        self.avg_pool = torch.nn.AdaptiveAvgPool2d(1)
        self.conv_du = torch.nn.Sequential(torch.nn.Conv2d(in_channels, d, 1, padding=0, bias=bias), torch.nn.LeakyReLU(0.2))

        self.fcs = torch.nn.ModuleList([])
        for i in range(self.height):
            self.fcs.append(torch.nn.Conv2d(d, in_channels, kernel_size=1, stride=1,bias=bias))

        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, inp_feats):
        batch_size = inp_feats[0].shape[0]
        n_feats =  inp_feats[0].shape[1]


        inp_feats = torch.cat(inp_feats, dim=1)
        inp_feats = inp_feats.view(batch_size, self.height, n_feats, inp_feats.shape[2], inp_feats.shape[3])

        feats_U = torch.sum(inp_feats, dim=1)
        feats_S = self.avg_pool(feats_U)
        feats_Z = self.conv_du(feats_S)

        attention_vectors = [fc(feats_Z) for fc in self.fcs]
        attention_vectors = torch.cat(attention_vectors, dim=1)
        attention_vectors = attention_vectors.view(batch_size, self.height, n_feats, 1, 1)
        # stx()
        attention_vectors = self.softmax(attention_vectors)

        feats_V = torch.sum(inp_feats*attention_vectors, dim=1)

        return feats_V

class ContextBlock(torch.nn.Module):

    def __init__(self, n_feat, bias=False):
        super(ContextBlock, self).__init__()

        self.conv_mask = torch.nn.Conv2d(n_feat, 1, kernel_size=1, bias=bias)
        self.softmax = torch.nn.Softmax(dim=2)

        self.channel_add_conv = torch.nn.Sequential(
            torch.nn.Conv2d(n_feat, n_feat, kernel_size=1, bias=bias),
            torch.nn.LeakyReLU(0.2),
            torch.nn.Conv2d(n_feat, n_feat, kernel_size=1, bias=bias)
        )

    def modeling(self, x):
        batch, channel, height, width = x.size()
        input_x = x
        # [N, C, H * W]
        input_x = input_x.view(batch, channel, height * width)
        # [N, 1, C, H * W]
        input_x = input_x.unsqueeze(1)
        # [N, 1, H, W]
        context_mask = self.conv_mask(x)
        # [N, 1, H * W]
        context_mask = context_mask.view(batch, 1, height * width)
        # [N, 1, H * W]
        context_mask = self.softmax(context_mask)
        # [N, 1, H * W, 1]
        context_mask = context_mask.unsqueeze(3)
        # [N, 1, C, 1]
        context = torch.matmul(input_x, context_mask)
        # [N, C, 1, 1]
        context = context.view(batch, channel, 1, 1)

        return context

    def forward(self, x):
        # [N, C, 1, 1]
        context = self.modeling(x)

        # [N, C, 1, 1]
        channel_add_term = self.channel_add_conv(context)
        x = x + channel_add_term

        return x

##########################################################################
### --------- Residual Context Block (RCB) ----------
class RCB(torch.nn.Module):
    def __init__(self, n_feat, kernel_size=3, reduction=8, bias=False, groups=1):
        super(RCB, self).__init__()

        act = torch.nn.LeakyReLU(0.2)

        self.body = torch.nn.Sequential(
            torch.nn.Conv2d(n_feat, n_feat, kernel_size=3, stride=1, padding=1, bias=bias, groups=groups),
            act,
            torch.nn.Conv2d(n_feat, n_feat, kernel_size=3, stride=1, padding=1, bias=bias, groups=groups)
        )

        self.act = act

        self.gcnet = ContextBlock(n_feat, bias=bias)

    def forward(self, x):
        res = self.body(x)
        res = self.act(self.gcnet(res))
        res += x
        return res


##########################################################################
##---------- Resizing Modules ----------
class Down(torch.nn.Module):
    def __init__(self, in_channels, chan_factor, bias=False):
        super(Down, self).__init__()

        self.bot = torch.nn.Sequential(
            torch.nn.AvgPool2d(2, ceil_mode=True, count_include_pad=False),
            torch.nn.Conv2d(in_channels, int(in_channels*chan_factor), 1, stride=1, padding=0, bias=bias)
            )

    def forward(self, x):
        return self.bot(x)

class DownSample(torch.nn.Module):
    def __init__(self, in_channels, scale_factor, chan_factor=2, kernel_size=3):
        super(DownSample, self).__init__()
        self.scale_factor = int(np.log2(scale_factor))

        modules_body = []
        for i in range(self.scale_factor):
            modules_body.append(Down(in_channels, chan_factor))
            in_channels = int(in_channels * chan_factor)

        self.body = torch.nn.Sequential(*modules_body)

    def forward(self, x):
        x = self.body(x)
        return x

class Up(torch.nn.Module):
    def __init__(self, in_channels, chan_factor, bias=False):
        super(Up, self).__init__()

        self.bot = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, int(in_channels//chan_factor), 1, stride=1, padding=0, bias=bias),
            torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=bias)
            )

    def forward(self, x):
        return self.bot(x)

class UpSample(torch.nn.Module):
    def __init__(self, in_channels, scale_factor, chan_factor=2, kernel_size=3):
        super(UpSample, self).__init__()
        self.scale_factor = int(np.log2(scale_factor))

        modules_body = []
        for i in range(self.scale_factor):
            modules_body.append(Up(in_channels, chan_factor))
            in_channels = int(in_channels // chan_factor)

        self.body = torch.nn.Sequential(*modules_body)

    def forward(self, x):
        x = self.body(x)
        return x


##########################################################################
##---------- Multi-Scale Resiudal Block (MRB) ----------
class MRB(torch.nn.Module):
    def __init__(self, n_feat, height, width, chan_factor, bias,groups):
        super(MRB, self).__init__()

        self.n_feat, self.height, self.width = n_feat, height, width

        self.dau_top = RCB(int(n_feat*chan_factor**0), bias=bias, groups=groups)
        self.dau_mid = RCB(int(n_feat*chan_factor**1), bias=bias, groups=groups)
        self.dau_bot = RCB(int(n_feat*chan_factor**2), bias=bias, groups=groups)

        self.down2 = DownSample(int((chan_factor**0)*n_feat),2,chan_factor)
        self.down4 = torch.nn.Sequential(
            DownSample(int((chan_factor**0)*n_feat),2,chan_factor),
            DownSample(int((chan_factor**1)*n_feat),2,chan_factor)
        )

        self.up21_1 = UpSample(int((chan_factor**1)*n_feat),2,chan_factor)
        self.up21_2 = UpSample(int((chan_factor**1)*n_feat),2,chan_factor)
        self.up32_1 = UpSample(int((chan_factor**2)*n_feat),2,chan_factor)
        self.up32_2 = UpSample(int((chan_factor**2)*n_feat),2,chan_factor)

        self.conv_out = torch.nn.Conv2d(n_feat, n_feat, kernel_size=1, padding=0, bias=bias)

        # only two inputs for SKFF
        self.skff_top = SKFF(int(n_feat*chan_factor**0), 2)
        self.skff_mid = SKFF(int(n_feat*chan_factor**1), 2)

    def forward(self, x):
        x_top = x.clone()
        x_mid = self.down2(x_top)
        x_bot = self.down4(x_top)

        x_top = self.dau_top(x_top)
        x_mid = self.dau_mid(x_mid)
        x_bot = self.dau_bot(x_bot)

        x_mid = self.skff_mid([x_mid, self.up32_1(x_bot)])
        x_top = self.skff_top([x_top, self.up21_1(x_mid)])

        x_top = self.dau_top(x_top)
        x_mid = self.dau_mid(x_mid)
        x_bot = self.dau_bot(x_bot)

        x_mid = self.skff_mid([x_mid, self.up32_2(x_bot)])
        x_top = self.skff_top([x_top, self.up21_2(x_mid)])

        out = self.conv_out(x_top)
        out = out + x

        return out

##########################################################################
##---------- Recursive Residual Group (RRG) ----------
class RRG(torch.nn.Module):
    def __init__(self, n_feat, n_MRB, height, width, chan_factor, bias=False, groups=1):
        super(RRG, self).__init__()
        modules_body = [MRB(n_feat, height, width, chan_factor, bias, groups) for _ in range(n_MRB)]
        modules_body.append(torch.nn.Conv2d(n_feat, n_feat, kernel_size=3, stride=1, padding=1, bias=bias))
        self.body = torch.nn.Sequential(*modules_body)

    def forward(self, x):
        res = self.body(x)
        res += x
        return res


##########################################################################
##---------- MIRNet  -----------------------
class MIRNet_v2(torch.nn.Module):
    def __init__(self,
        inp_channels=3,
        out_channels=3,
        n_feat=80,
        chan_factor=1.5,
        n_RRG=4,
        n_MRB=2,
        height=3,
        width=2,
        scale=1,
        bias=False,
        task= None
    ):
        super(MIRNet_v2, self).__init__()

        kernel_size=3
        self.task = task

        self.conv_in = torch.nn.Conv2d(inp_channels, n_feat, kernel_size=3, padding=1, bias=bias)

        modules_body = []

        modules_body.append(RRG(n_feat, n_MRB, height, width, chan_factor, bias, groups=1))
        modules_body.append(RRG(n_feat, n_MRB, height, width, chan_factor, bias, groups=2))
        modules_body.append(RRG(n_feat, n_MRB, height, width, chan_factor, bias, groups=4))
        modules_body.append(RRG(n_feat, n_MRB, height, width, chan_factor, bias, groups=4))

        self.body = torch.nn.Sequential(*modules_body)
        self.conv_out = torch.nn.Conv2d(n_feat, out_channels, kernel_size=3, padding=1, bias=bias)


    def forward(self, inp_img):
        shallow_feats = self.conv_in(inp_img)
        deep_feats = self.body(shallow_feats)

        if self.task == 'defocus_deblurring':
            deep_feats += shallow_feats
            out_img = self.conv_out(deep_feats)

        else:
            out_img = self.conv_out(deep_feats)
            out_img += inp_img

        return out_img


In [ ]:
class CharbonnierLoss(torch.nn.Module):
    """Charbonnier Loss (L1)"""

    def __init__(self, eps:float = 1e-3):
        super(CharbonnierLoss, self).__init__()
        self.eps = eps

    def forward(self, x, y):
        diff = x - y
        loss = torch.mean(torch.sqrt((diff ** 2) + (self.eps ** 2)))
        return loss

In [ ]:
class PairedRandomCrop(torch.nn.Module):
  def __init__(self, size:torch.IntTensor):
    super(PairedRandomCrop, self).__init__()
    self.size:torch.IntTensor = size
  def forward(self, data_list:list[torch.Tensor]):
    location:torch.IntTensor = (torch.rand(2) * (torch.IntTensor(list(data_list[0].size())[1:]) - self.size)).to(dtype = torch.int)
    new_list:list[torch.Tensor] = [torchvision.transforms.functional.crop(data, location[0], location[1], self.size[0], self.size[1]) for data in data_list]
    return new_list

In [ ]:
class LOLDataset(torch.utils.data.Dataset):
	def __init__(self, root_dir:str, sub_dir:list[str], prefix:list[str] = ["", ""], transform = None) -> None:
		super().__init__()
		self.root_dir:str = root_dir
		self.sub_dir:list[str] = sub_dir
		self.prefix:list[str] = prefix
		self.transform = transform
		self.file_name:list[str] = sorted(os.listdir(os.path.join(self.root_dir, self.sub_dir[0])))
		self.file_name:list[str] = [fn.removeprefix(self.prefix[0]) for fn in self.file_name if fn.endswith(".png")]
	def __len__(self) -> int:
		return len(self.file_name)
	def __getitem__(self, idx:int) -> list[torch.FloatTensor]:
		file_path:list[str] = [os.path.join(self.root_dir, sd, pf + self.file_name[idx]) for sd, pf in zip(self.sub_dir, self.prefix)]
		data:list[np.array] = [torch.as_tensor(cv2.imread(fp)).float() for fp in file_path]
		data:list[np.array] = [np.transpose(d, [2, 0, 1]) for d in data]
		data:list[torch.Tensor] = [torch.as_tensor(d) for d in data]
		data:list[torch.FloatTensor] = [d.float() for d in data]
		if self.transform != None:
			data = self.transform(data)
		return data

In [ ]:
class BGR2LAB(torch.nn.Module):
  def __init__(self) -> None:
    super(BGR2LAB, self).__init__()
  def forward(self, data:torch.Tensor) -> torch.Tensor:
    data:torch.Tensor = data.permute(0, 2, 3, 1)
    data:list[np.ndarray] = [img.detach().cpu().numpy() for img in data]
    data:list[np.ndarray] = [cv2.cvtColor(img, cv2.COLOR_BGR2LAB) for img in data]
    data:np.ndarray = np.array([img.tolist() for img in data])
    data:np.ndarray = np.transpose(data, [0, 3, 1, 2])
    data:torch.Tensor = torch.as_tensor(data)
    return data

In [ ]:
def seed_everything(seed: int = 42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.cuda.manual_seed_all(seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore

In [ ]:
def train(dataset:list[torch.utils.data.Dataset], device:str):
    epochs = 64
    batch_size = 2
    loader:list[torch.utils.data.DataLoader] = [
      torch.utils.data.DataLoader(dataset[0], batch_size = batch_size, shuffle = True),
      torch.utils.data.DataLoader(dataset[1], batch_size = 1, shuffle = False),
    ]
    model:torch.nn.Module = MIRNet_v2().to(device = device)
    bgr2lab:torch.nn.Module = BGR2LAB().to(device = device)
    criterion:torch.nn.Module = CharbonnierLoss().to(device = device)
    optimizer:torch.optim.Optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler:torch.optim.lr_scheduler.LRScheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-5)
    metric:list[torchmetrics.metric.Metric] = [
        torchmetrics.image.PeakSignalNoiseRatio().to(device = device),
        torchmetrics.image.StructuralSimilarityIndexMeasure().to(device = device),
    ]

    best_val_loss:float = cmath.inf

    for epoch in range(1, epochs + 1):
        sum_loss:list[float] = [0.0 for _ in range(2)]
        sum_metric:list[float] = [0.0 for m in metric]

        loop = tqdm(enumerate(loader[0]), total = len(loader[0]), leave = False)

        model.train()
        for idx, data in loop:
            optimizer.zero_grad()
            data:list[torch.Tensor] = [d.to(device = device) for d in data]
            data[0]:torch.Tensor = model(data[0])
            #data:list[torch.Tensor] = [bgr2lab(d) for d in data]
            loss:torch.Tensor = criterion(data[0], data[1])
            loss.backward()
            optimizer.step()

            sum_loss[0] += loss.item()
            loop.set_description(f"Epoch {epoch}/{epochs}")

        current_lr = scheduler.optimizer.param_groups[0]["lr"]
        scheduler.step()

        model.eval()
        with torch.no_grad():
            loop = tqdm(enumerate(loader[1]), total = len(loader[1]), leave = False)

            for idx, data in loop:
                data:list[torch.Tensor] = [d.to(device = device) for d in data]
                data[0]:torch.Tensor = model(data[0])
                #data:list[torch.Tensor] = [bgr2lab(d) for d in data]
                loss:torch.Tensor = criterion(data[0], data[1])
                sum_loss[1] += loss.item()
                for i in range(len(metric)):
                  sum_metric[i] += metric[i](data[0], data[1]).item()
                loop.set_description("valid")

        avg_loss:list[float] = [sum_loss[i] / len(loader[i]) for i in range(2)]
        avg_metric:list[float] = [s / len(loader[1]) for s in sum_metric]

        print(f"Epoch: {epoch}\ttrain_loss: {avg_loss[0]:.4f}\tval_loss: {avg_loss[1]:.4f}\tpsnr: {avg_metric[0]:.4f}\tssim: {avg_metric[1]:.4f}")

        if best_val_loss > avg_loss[1]:
            print("=" * 80)
            print(f"val_loss is improved from {best_val_loss:.4f} to {avg_loss[1]:.4f}\t saved current weight")
            print("=" * 80)
            best_val_loss:float = avg_loss[1]

            torch.save(
              {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": criterion
              },
              os.path.join("drive", "My Drive", "llie", "weight", f"{epoch:03d}.pth")
            )

In [ ]:
def test(dataset:torch.utils.data.Dataset, device:str):
  loader:torch.utils.data.DataLoader = torch.utils.data.DataLoader(dataset, batch_size = 1)
  last_cp_path:str = sorted(os.listdir(os.path.join("drive", "My Drive", "llie", "weight")))[-1]
  cp:dict = torch.load(os.path.join("drive", "My Drive", "llie", "weight", last_cp_path))
  model:torch.nn.Module = MIRNet_v2().to(device = device)
  criterion:torch.nn.Module = CharbonnierLoss().to(device = device)
  metric:list[torchmetrics.metric.Metric] = [
    torchmetrics.image.PeakSignalNoiseRatio().to(device = device),
    torchmetrics.image.StructuralSimilarityIndexMeasure().to(device = device),
  ]
  model.load_state_dict(cp["model_state_dict"])
  model.eval()
  with torch.no_grad():
    loop = tqdm(enumerate(loader), total = len(loader), leave = False)
    for idx, data in loop:
      data:list[torch.Tensor] = [d.to(device = device) for d in data]
      data.append(model(data[0]))
      loss:torch.Tensor = criterion(data[2], data[1])
      metric_val:list[float] = [m(data[2], data[1]).item() for m in metric]
      print(f"idx: {idx}\tloss: {loss.item():.4f}\tpsnr: {metric_val[0]:.4f}\tssim: {metric_val[1]:.4f}")
      for i, d in enumerate(data):
        try:
          os.mkdir(os.path.join("drive", "My Drive", "llie", "img", str(i)))
        except:
          pass
        for img in d.permute(0, 2, 3, 1).detach().cpu().numpy():
          cv2.imwrite(os.path.join("drive", "My Drive", "llie", "img", str(i), f"{idx:03d}.png"), img)
      loop.set_description("test")

In [ ]:
if __name__ == "__main__":
  device:str = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
  )
  transform:torch.nn.Module = PairedRandomCrop(torch.IntTensor([360, 540]))
  dataset:list[torch.utils.data.Dataset] = [
    LOLDataset(root_dir = os.path.join("dataset", "LOL-v2", "Real_captured", "Train"), sub_dir = ["Low", "Normal"], prefix = ["low", "normal"]),
    LOLDataset(root_dir = os.path.join("dataset", "LOL-v2", "Real_captured", "Test"), sub_dir = ["Low", "Normal"], prefix = ["low", "normal"]),
    LOLDataset(root_dir = os.path.join("dataset", "lol_dataset", "our485"), sub_dir = ["low", "high"]),
    LOLDataset(root_dir = os.path.join("dataset", "lol_dataset", "eval15"), sub_dir = ["low", "high"]),
  ]
  seed_everything(42)
  google.colab.drive.mount("drive")
  train(dataset[0:2], device)
  test(dataset[3], device)
  google.colab.drive.flush_and_unmount()

Mounted at drive


Epoch: 1	train_loss: 46.7869	val_loss: 24.9756	psnr: 18.2437	ssim: 0.8292
val_loss is improved from inf to 24.9756	 saved current weight


Epoch: 2	train_loss: 33.1583	val_loss: 21.8736	psnr: 19.9133	ssim: 0.8255
val_loss is improved from 24.9756 to 21.8736	 saved current weight


Epoch: 3	train_loss: 30.0386	val_loss: 24.9160	psnr: 19.2515	ssim: 0.8164


Epoch: 4	train_loss: 26.4952	val_loss: 30.6292	psnr: 17.9175	ssim: 0.8026


...

idx: 12	loss: 18.8952	psnr: 20.3908	ssim: 0.6994


test:  87%|████████▋ | 13/15 [00:40<00:05,  2.87s/it]

idx: 13	loss: 17.3078	psnr: 21.4421	ssim: 0.7760


test:  93%|█████████▎| 14/15 [00:43<00:02,  2.82s/it]

idx: 14	loss: 51.2711	psnr: 13.6379	ssim: 0.8602
